<h1>3-Matrix-Factorization</h1>

<h3><a href="https://www.kaggle.com/sengzhaotoo/movielens-small">DataSet</a></h3>

<h2>پیش پردازش داده ها</h2>
<h2>PreProcessing</h2>

In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
rating_data = pd.read_csv('datasets/movie-lens/ratings.csv', low_memory=False)
movie_data = pd.read_csv('datasets/movie-lens/movies.csv', low_memory=False)
rating_data.head(4)

In [ ]:
movie_data.head()

In [ ]:
print(movie_data.shape)
print(rating_data.shape)

In [ ]:
rating_data.drop('timestamp', axis = 1, inplace = True)
rating_data.head()

In [ ]:
movie_data.drop('genres', axis = 1, inplace = True)
movie_data.head()

In [ ]:
user_movie_data = pd.merge(rating_data, movie_data, on = 'movieId')
user_movie_data.head()

In [ ]:
user_movie_data.shape

In [13]:
user_movie_rating = user_movie_data.pivot_table('rating', index = 'userId', columns='title').fillna(0)

In [ ]:
user_movie_rating.shape

In [ ]:
user_movie_rating.head()

In [ ]:
movie_user_rating = user_movie_rating.values.T
movie_user_rating.shape

In [ ]:
type(movie_user_rating)

<h1>SVD(Singular Value Decomposion)</h1>

In [ ]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(movie_user_rating)
matrix.shape

In [ ]:
matrix[0]

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

In [ ]:
corr2 = corr[:200, :200]
corr2.shape

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(corr2)

In [23]:
movie_title = user_movie_rating.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Guardians of the Galaxy (2014)")

In [ ]:
corr_coffey_hands  = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]

In [25]:
df_ratings = pd.read_csv('datasets/movie-lens/ratings.csv', low_memory=False)
df_movies =  pd.read_csv('datasets/movie-lens/movies.csv', low_memory=False)

In [26]:
df_user_movie_ratings = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [ ]:
df_user_movie_ratings.head()

In [28]:
matrix = df_user_movie_ratings.values
# میانگین امتیاز کاربر
user_ratings_mean = np.mean(matrix, axis = 1)
# R_user_mean: میانگین امتیاز کاربر منهای میانگین امتیاز کاربر برای فیلم.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [ ]:
matrix

In [ ]:
user_ratings_mean.shape

In [ ]:
matrix_user_mean.shape

In [ ]:
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

In [33]:
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [ ]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

In [35]:
sigma = np.diag(sigma)

In [ ]:
sigma.shape

In [ ]:
sigma[0]

In [ ]:
sigma[1]

In [39]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds.head()

In [ ]:
df_svd_preds.shape

In [42]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    
    user_row_number = user_id - 1 
    
    
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
     
    user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
    
    
    user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending=False)
    
    
    recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]
    
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
    
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [43]:
already_rated, predictions = recommend_movies(df_svd_preds, 330, df_movies, df_ratings, 10)

In [ ]:
already_rated.head(10)

In [ ]:
predictions

<h1>Level Of Code : Basic</h1>